#Setup

##Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import sys
sys.path.append('/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch')

import os
from image import *
from model import CSRNet
import glob
import torch

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##Variables
Paramaters to change according to the wanted output

In [ ]:
#parameters to change
INPUT_MODEL_NAME = 'A10_SH_C.tar' #Name of the saved CSRNet model weight
OUTPUT_MODEL_NAME = 'A10_SH_512x384' # Will be directory with the json and the bins, usable with tensorflowJS
Input_img_size = (512,384) #Size of the model predictions
use_gpu = False

#paths

In [ ]:
root = '/content/gdrive/MyDrive/TFE_crowd_counting'

scripts_path = os.path.join(root,'CSRNet-pytorch')
models_path = os.path.join(scripts_path,'models')
onnx_path = os.path.join(scripts_path,'models/ONNX')
input_model_path = os.path.join(models_path,INPUT_MODEL_NAME)
onnx_model_path = os.path.join(onnx_path, OUTPUT_MODEL_NAME+'.onnx')
tf_model_path = os.path.join(models_path, OUTPUT_MODEL_NAME+'.pb')
output_path = os.path.join(models_path, OUTPUT_MODEL_NAME)

#Utils

In [ ]:
#return the loaded model located at model_path (or the basic shangai partAmodel if no path is given ) 
#using gpu or using cpu (if the use_gpu parameter is set to False)
def load_model(model_path, use_gpu = use_gpu):
  from torchvision import datasets, transforms
  transform=transforms.Compose([
                        transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
                    ])
  model = CSRNet()
  checkpoint = torch.load(model_path)
  model.load_state_dict(checkpoint['state_dict'])
  if use_gpu:
    model = model.cuda()
  else:
    model = model.cpu()
  return model

#Conversion

In [ ]:
#Step 1: convert to onnx
use_gpu = False
model = load_model(input_model_path)
input_names = [ "image" ]
output_names = [ "output1" ]
if use_gpu:
  dummy_input = torch.randn(1, 3, Input_img_size[0], Input_img_size[1], device='cuda')
else:
  dummy_input = torch.randn(1, 3, Input_img_size[1], Input_img_size[0])
torch.onnx.export(model, dummy_input, onnx_model_path, operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)

/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:286: UserWarning: `add_node_names' can be set to True only when 'operator_export_type' is `ONNX`. Since 'operator_export_type' is not set to 'ONNX', `add_node_names` argument will be ignored.
  "`{}` argument will be ignored.".format(arg_name, arg_name))
/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:286: UserWarning: `do_constant_folding' can be set to True only when 'operator_export_type' is `ONNX`. Since 'operator_export_type' is not set to 'ONNX', `do_constant_folding` argument will be ignored.
  "`{}` argument will be ignored.".format(arg_name, arg_name))


In [ ]:
#Step 2: export to tensorflow
#!pip install git+https://github.com/onnx/onnx-tensorflow.git #May be commented if the package is already installed
!onnx-tf convert -i {onnx_model_path} -o {tf_model_path}

2022-05-20 12:21:03,532 - onnx-tf - INFO - Start converting onnx pb to tf saved model
2022-05-20 12:21:04.337940: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-05-20 12:21:10.971681: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-05-20 12:21:12,253 - onnx-tf - INFO - Converting completes successfully.
INFO:onnx-tf:Converting completes successfully.


In [ ]:
#Step 3: export to tensorflow.js
#!pip install tensorflowjs #May be commented if the package is already installed

In [ ]:
#Copy past the two path as argument for conversion
print(tf_model_path)
print(output_path)

/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/models/A10_SH_256x192.pb
/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/models/A10_SH_256x192


In [ ]:
!tensorflowjs_converter --input_format='tf_saved_model' '/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/models/A10_SH_512x384.pb' '/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/models/A10_SH_512x384'

2022-05-20 12:21:24.979366: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Writing weight file /content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/models/A10_SH_512x384/model.json...
